# Preparing kaggle Titanic dataset

## Import necessary libraries

In [320]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## load the data

In [321]:
raw_df = pd.read_csv('data/train.csv')
df = raw_df.copy()

In [322]:
print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Drop the features 'PassengerId' and 'Name' to make the data easier and more concise. (for now)

In [323]:
df = df.drop(['PassengerId', 'Name'], axis=1)

_Note: We can potentially use the names of the passengers to infer their family relations, which might be useful for predicting their survival. For example if lot of passengers share the same last name, they might be family members and help each other to survive._

**Quick table visualization of the first 10 rows**

In [324]:
df[:10].style.format(precision=2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.00,1,0,A/5 21171,7.25,nan,S
1,1,1,female,38.00,1,0,PC 17599,71.28,C85,C
2,1,3,female,26.00,0,0,STON/O2. 3101282,7.92,nan,S
3,1,1,female,35.00,1,0,113803,53.10,C123,S
4,0,3,male,35.00,0,0,373450,8.05,nan,S
5,0,3,male,nan,0,0,330877,8.46,nan,Q
6,0,1,male,54.00,0,0,17463,51.86,E46,S
7,0,3,male,2.00,3,1,349909,21.07,nan,S
8,1,3,female,27.00,0,2,347742,11.13,nan,S
9,1,2,female,14.00,1,0,237736,30.07,nan,C


## Encoding categorical features

### Finding number of unique values for each categorical feature

In [325]:
categorical_features = ['Pclass', 'Sex', 'Cabin', 'Embarked']

for feature in categorical_features:
    print(f'{feature}: {df[feature].nunique()}')

Pclass: 3
Sex: 2
Cabin: 147
Embarked: 3


### Preparing Cabin feature

Cabin is a categorical feature, but it has too many unique values.

In [326]:
cabin_unique = df['Cabin'].dropna().unique()
print(f'Number of unique values for Cabin: {len(cabin_unique)}')
print(cabin_unique)

Number of unique values for Cabin: 147
['C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33' 'B30'
 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49'
 'F4' 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77'
 'E67' 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106'
 'C65' 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91'
 'E40' 'T' 'C128' 'D37' 'B35' 'E50' 'C82' 'B96 B98' 'E10' 'E44' 'A34'
 'C104' 'C111' 'C92' 'E38' 'D21' 'E12' 'E63' 'A14' 'B37' 'C30' 'D20' 'B79'
 'E25' 'D46' 'B73' 'C95' 'B38' 'B39' 'B22' 'C86' 'C70' 'A16' 'C101' 'C68'
 'A10' 'E68' 'B41' 'A20' 'D19' 'D50' 'D9' 'A23' 'B50' 'A26' 'D48' 'E58'
 'C126' 'B71' 'B51 B53 B55' 'D49' 'B5' 'B20' 'F G63' 'C62 C64' 'E24' 'C90'
 'C45' 'E8' 'B101' 'D45' 'C46' 'D30' 'E121' 'D11' 'E77' 'F38' 'B3' 'D6'
 'B82 B84' 'D17' 'A36' 'B102' 'B69' 'E49' 'C47' 'D28' 'E17' 'A24' 'C50'
 'B42' 'C148']


To reduce the number of unique values in Cabin, we can split the cabin number into two parts:
* The first letter is the deck of the cabin, which is a **categorical feature**. 
* The number is the room number, which is a **numerical feature**.

Also some cabins have multiple rooms, so we will only use the first room number.

For example the cabin number 'C85' will be split into: deck 'C' ,room '85'.

*One way to think about it is that the deck is the z coordinate since decks are stacked on top of each other, and the room number is the x coordinate since rooms are arranged horizontally.*

In [327]:
# chose only first cabin if there are multiple
df['Cabin'] = df['Cabin'].apply(lambda s: s.split(' ')[0] if pd.notnull(s) else np.nan)

# extract deck and room from cabin
df['Deck'] = df['Cabin'].apply(lambda s: s[0] if pd.notnull(s) else np.nan)
df['Room'] = df['Cabin'].apply(lambda s: int(s[1:]) if pd.notnull(s) and s[1:] != '' else np.nan)

# drop cabin since its no longer needed
df = df.drop('Cabin', axis=1)

# visualize the first 5 rows
df[:5].style.format(precision=2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Deck,Room
0,0,3,male,22.00,1,0,A/5 21171,7.25,S,nan,nan
1,1,1,female,38.00,1,0,PC 17599,71.28,C,C,85.00
2,1,3,female,26.00,0,0,STON/O2. 3101282,7.92,S,nan,nan
3,1,1,female,35.00,1,0,113803,53.10,S,C,123.00
4,0,3,male,35.00,0,0,373450,8.05,S,nan,nan


Verifying the new features.

In [328]:
deck_unique = df['Deck'].dropna().unique()
print(f'Number of unique values for Deck: {len(deck_unique)}')
print(deck_unique)
print()

room_max = df['Room'].max()
room_min = df['Room'].min()
print(f'Room max: {room_max}')
print(f'Room min: {room_min}')

Number of unique values for Deck: 8
['C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']

Room max: 148.0
Room min: 2.0


### Preparing Ticket feature

In [329]:
df[['Ticket']][:5]

,Ticket
0,A/5 21171
1,PC 17599
2,STON/O2. 3101282
3,113803
4,373450


The feature Ticket can be used as numerical feature if we extract the ticket number from it.

This might help in indicating the booking order of the passengers.

In [330]:
df['Ticket'] = df['Ticket'].map(lambda s:s.split(' ')[-1])
df[['Ticket']][:5]

,Ticket
0,21171
1,17599
2,3101282
3,113803
4,373450


In [331]:
# show type of ticket column
df['Ticket'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: Ticket
Non-Null Count  Dtype 
--------------  ----- 
891 non-null    object
dtypes: object(1)
memory usage: 7.1+ KB


Seems like some tickets have a numerical prefix, and some don't, which led to some values being of type object

In [332]:
# show only non numeric tickets
df[df['Ticket'].str.isnumeric() == False][['Ticket']]

,Ticket
179,LINE
271,LINE
302,LINE
597,LINE


Since there are only few tickets that don't have a numerical prefix, we can just replace them with nan.

In [333]:
# replace all non numeric tickets with nan
df['Ticket'] = df['Ticket'].apply(lambda s: s if s.isnumeric() else np.nan)
# convert ticket to numeric
df['Ticket'] = pd.to_numeric(df['Ticket'])

In [334]:
# show type of ticket column
df['Ticket'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: Ticket
Non-Null Count  Dtype  
--------------  -----  
887 non-null    float64
dtypes: float64(1)
memory usage: 7.1 KB


Now the type of Ticket is float, 

### Hot-one encoding categorical features

After reducing the number of unique values for Cabin, we can now use hot one encoding to encode all categorical features.

In [335]:
# update new categorical features after processing Cabin
categorical_features.remove('Cabin')
categorical_features.append('Deck')

# transform Pclass to string to be treated as categorical feature
df['Pclass'] = df['Pclass'].astype(str)

# hot-one encoding all categorical features
dummies = pd.get_dummies(df[categorical_features])

# drop the original categorical features
df = df.drop(categorical_features, axis=1)

# add the new hot-one encoded features
df = pd.concat([df, dummies], axis=1)

### Final result

In [336]:
# visualize the first 5 rows
df[:5].style.format(precision=2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Age         714 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Parch       891 non-null    int64  
 4   Ticket      887 non-null    float64
 5   Fare        891 non-null    float64
 6   Room        196 non-null    float64
 7   Pclass_1    891 non-null    uint8  
 8   Pclass_2    891 non-null    uint8  
 9   Pclass_3    891 non-null    uint8  
 10  Sex_female  891 non-null    uint8  
 11  Sex_male    891 non-null    uint8  
 12  Embarked_C  891 non-null    uint8  
 13  Embarked_Q  891 non-null    uint8  
 14  Embarked_S  891 non-null    uint8  
 15  Deck_A      891 non-null    uint8  
 16  Deck_B      891 non-null    uint8  
 17  Deck_C      891 non-null    uint8  
 18  Deck_D      891 non-null    uint8  
 19  Deck_E      891 non-null    u

## Pack all the preprocessing steps into a function

To make it easier to preprocess the test data, we will pack all the preprocessing steps into a function.

In [337]:
from utils import preprocess_data

Testing the function on the same dataset as before.

In [338]:
df = raw_df.copy()
df = preprocess_data(df)
df[:5].style.format(precision=2)

,Survived,Age,SibSp,Parch,Ticket,Fare,Room,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
0,0,22.00,1,0,21171,7.25,nan,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
1,1,38.00,1,0,17599,71.28,85.00,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,1,26.00,0,0,3101282,7.92,nan,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
3,1,35.00,1,0,113803,53.10,123.00,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0
4,0,35.00,0,0,373450,8.05,nan,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0


## Splitting the data

Here we will split the data into training, validation and test sets.

In [339]:
from utils import split_data


X_train, y_train, X_val, y_val, X_test, y_test = split_data(df)

In [340]:
print(f'Train size: {len(X_train)} ({len(X_train)/len(df)*100:.2f}%)')
print(f'Validation size: {len(X_val)} ({len(X_val)/len(df)*100:.2f}%)')
print(f'Test size: {len(X_test)} ({len(X_test)/len(df)*100:.2f}%)')

Train size: 712 (79.91%)
Validation size: 89 (9.99%)
Test size: 90 (10.10%)


In [341]:
X_train[:5].style.format(precision=2)

,Age,SibSp,Parch,Ticket,Fare,Room,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
58,5.00,1,2,34651,27.75,nan,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0
314,43.00,1,1,13529,26.25,nan,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0
365,30.00,0,0,7076,7.25,nan,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
700,18.00,1,0,17757,227.53,62.00,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0
408,21.00,0,0,312992,7.78,nan,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0


In [342]:
y_train.iloc[:5]

58     1
314    0
365    0
700    1
408    0
Name: Survived, dtype: int64

## Conclusion

Finally we're done with the preprocessing the data is ready to be fed into a machine learning model.

The next step is to choose a model and train it on the training set.